In [22]:
# Решето Эратосфена
def generate_prime_table(n):
    multiples = set()
    for i in range(2, n+1):
        if i not in multiples:
            yield i
            multiples.update(range(i*i, n+1, i))

# subexponentional funtion
def L(c, g, q): return e^(c*(log(q))^g *(log(log(q)))^(1-g))

In [23]:
def check_factor(number, primeTable):
    # we should not use only factor function
    # 1. we should try Miller-Rabin test [TODO]
    # 2. we should try Brenet's algoritm (or its analog like Lenstra alg) [TODO]
    #-----------------------------------
    decomposition = dict(factor(Integer(number)))
    # ----------------------------------
    numberPrimeFactors = set(decomposition.keys())
    success = (numberPrimeFactors.issubset(primeTable))
    
    return (success, decomposition)
        

def generate_equasion(a, q, primeTable, ring):
    
    equasionGenerated = False
    while not equasionGenerated:
        xsi = randint(2, q-1)
        axsi = ring(a)^xsi
        equasionGenerated, decomposition = check_factor(axsi, primeTable)
        
    coefficients = [decomposition[p] if p in decomposition else 0 for p in primeTable]
    
    return (xsi, coefficients)

In [26]:
def adleman_log(a, b, p): # return log_a (b) (mod p)
    if (p not in Primes()):
        return None
    
    ring = IntegerModRing(p)
    q = ring(a).multiplicative_order()
    
    # TO DO: when q=p1^k1*...*ps^k2
    #        use Chinese remainder theorem
    if (q not in Primes()):
        return None
    
    border = ceil(L(1/2, 1/2, q)) #it is taken like this because of complexity balance
    primeTable = set(generate_prime_table(border))
    
    # generating the equasions
    matrixGenerated = False
    remainedEquations = len(primeTable)
    equasions = []
    while not matrixGenerated: 
        equasions += [generate_equasion(a,q,primeTable,ring) for _ in range(remainedEquations)]
        coefficients = list(map(lambda x: x[1], equasions))
        
        # USE Chinese remainder theorem SOME HOWE
        linearIndependentColumns = Matrix(IntegerModRing(q), coefficients).pivots()
        
        
        remainedEquations = len(primeTable) - len(linearIndependentColumns)
        matrixGenerated = (remainedEquations == 0)
        
        equasions = [ equasions[i] for i in linearIndependentColumns]
        
    return equasions
    # at this point equasions are generated, so we're solving them
    y = vector(list(map(lambda x: x[0], equasions)))
    
    # IntegerModRing(ord(a))
    A = Matrix(ring, list(map(lambda x: x[1], equasions)))
    x = (A^-1) * y
    
    # finding log_a (b)
    while True:
        xsi = randint(2, q-1)
        bxsi = ring(b)^xsi
        found, decomposition = check_factor(bxsi, primeTable)
        if found:
            
            b_coeffisients = vector([decomposition[p] if p in decomposition else 0 for p in primeTable])
            
            return (ring(xsi)^-1)*(x.dot_product(b_coeffisients))

In [27]:
adleman_log(15, 23, 65521)

In [7]:
a,b,p = 15, 23, 65521
ring = IntegerModRing(p)
q = ring(a).multiplicative_order()
border = ceil(L(1/2, 1/2, q))
primeTable = set(generate_prime_table(border))

In [8]:
len(primeTable)

3

In [9]:
axis = set([ring(a)^i for i in range(q)])

coefs_all = []
for ax in axis:
    dec = dict(factor(Integer(ax)))
    numberPrimeFactors = set(dec.keys())
    if (not numberPrimeFactors.issubset(primeTable)):
        continue
    coefs_all += [[dec[p] if p in dec else 0 for p in primeTable]]

In [12]:
Matrix(coefs_all).pivots()

(0, 1)

In [19]:
Matrix(IntegerModRing(q), coefs_all).pivots()

NotImplementedError: Echelon form not implemented over 'Ring of integers modulo 585'.

In [14]:
coefs_all

[[14, 0, 0],
 [0, 0, 0],
 [12, 0, 0],
 [12, 1, 1],
 [2, 0, 0],
 [0, 1, 1],
 [4, 0, 0],
 [2, 1, 1],
 [6, 0, 0],
 [6, 2, 2],
 [0, 2, 2],
 [4, 1, 1],
 [8, 0, 0],
 [8, 2, 2],
 [4, 3, 3],
 [2, 2, 2],
 [6, 1, 1],
 [10, 0, 0],
 [10, 1, 1],
 [2, 3, 3],
 [0, 3, 3],
 [0, 4, 4],
 [4, 2, 2],
 [8, 1, 1]]

In [51]:
ring = IntegerModRing(p)
q = ring(a).multiplicative_order()
border = ceil(L(1/2, 1/2, q)) #it is taken like this because of complexity balance
primeTable = set(generate_prime_table(border))

In [52]:
matrixGenerated = False
remainedEquations = len(primeTable)
equasions = []
while not matrixGenerated: 
    equasions += [generate_equasion(a,q,primeTable,ring) for _ in range(remainedEquations)]
    coefficients = list(map(lambda x: x[1], equasions))
    print(equasions)
    print(Matrix(ring, coefficients))
    linearIndependentColumns = Matrix(ring, coefficients).pivots()
    remainedEquations = len(primeTable) - len(linearIndependentColumns)
    matrixGenerated = (remainedEquations == 0)   
    equasions = [ equasions[i] for i in linearIndependentColumns]

[(3, [3, 3]), (515, [2, 3, 3]), (4, [0, 0])]


ValueError: inconsistent number of columns: should be 2 but got 3

In [56]:
equasionGenerated = False
decomposition = []

while not equasionGenerated:
    xsi = randint(2, q-1)
    axsi = ring(a)^xsi
    equasionGenerated, decomposition = check_factor(axsi, primeTable)
    
coefficients = list(map(lambda x: x[1] if x[1] in primeTable else 0, decomposition))

(xsi, coefficients)

(368, [0, 2, 2])

In [30]:
r = IntegerModRing(65521)
factor(r(254))

TypeError: unable to factor 254

In [71]:
xsi = 3
equasionGenerated, decomposition = check_factor(ring(a)^xsi, primeTable)

In [89]:
decomposition = dict(decomposition)
coefficients = [decomposition[p] if p in decomposition else 0 for p in primeTable]
coefficients

[0, 3, 3]

In [85]:
set(dict(decomposition).keys())

{3, 5}

False